# Dino for perceptual loss

Download the backbones

In [ ]:
import torch

dinov2_vits14 = torch.hub.load('facebookresearch/dinov2', 'dinov2_vits14')
#dinov2_vitb14 = torch.hub.load('facebookresearch/dinov2', 'dinov2_vitb14')
#dinov2_vitl14 = torch.hub.load('facebookresearch/dinov2', 'dinov2_vitl14')
#dinov2_vitg14 = torch.hub.load('facebookresearch/dinov2', 'dinov2_vitg14')

Using cache found in /home/rtxmsi1/.cache/torch/hub/facebookresearch_dinov2_main
/home/rtxmsi1/.cache/torch/hub/facebookresearch_dinov2_main/dinov2/layers/swiglu_ffn.py:51: UserWarning: xFormers is not available (SwiGLU)
  warnings.warn("xFormers is not available (SwiGLU)")
/home/rtxmsi1/.cache/torch/hub/facebookresearch_dinov2_main/dinov2/layers/attention.py:33: UserWarning: xFormers is not available (Attention)
  warnings.warn("xFormers is not available (Attention)")
/home/rtxmsi1/.cache/torch/hub/facebookresearch_dinov2_main/dinov2/layers/block.py:40: UserWarning: xFormers is not available (Block)
  warnings.warn("xFormers is not available (Block)")


Read the image

In [8]:
from torchvision import transforms as pth_transforms
from PIL import Image

img = Image.open("/home/rtxmsi1/Documents/DINO/src/Vd-Orig.png")
img = img.convert('RGB')
transform = pth_transforms.Compose([
        pth_transforms.Resize(126),
        pth_transforms.ToTensor(),
        pth_transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225)),
    ])

img = transform(img)
img.shape

torch.Size([3, 126, 126])

### Using CLS token

Pros: Designed as a global summary

In [9]:
# https://github.com/facebookresearch/dino/blob/main/eval_linear.py#L153
avgpool = False
n = 10

with torch.no_grad():
    if "vit":
        intermediate_output = dinov2_vits14.get_intermediate_layers(img.unsqueeze(0), n)
        # Get the CLS token for each intermediate output
        output = torch.cat([x[:, 0] for x in intermediate_output], dim=-1)
        if avgpool:
            output = torch.cat((output.unsqueeze(-1), torch.mean(intermediate_output[-1][:, 1:], dim=1).unsqueeze(-1)), dim=-1)
            output = output.reshape(output.shape[0], -1)
    else:
        output = dinov2_vits14(img)

output.shape

torch.Size([1, 3840])

In [10]:
import torch

x = (torch.rand(1, 3840) * 20) - 10  # Scale [0,1) → [0,20) → [-10,10)
print(x.shape)  # torch.Size([1, 3840])
print(x.min(), x.max())  # Should be near -10 and 10

torch.Size([1, 3840])
tensor(-9.9938) tensor(9.9907)


In [11]:
import torch.nn.functional as F
mse = F.mse_loss(output, x, reduction="sum")
mse

tensor(193096.2031)

### Mean‑pooled tokens

What: average over all tokens (patch embeddings)

In [12]:
intermediate_output[1].mean(dim=1).shape

mse = F.mse_loss(intermediate_output[1].mean(dim=1), intermediate_output[1].mean(dim=1), reduction="sum")
mse

tensor(0.)

### Full token map (spatial loss)
Pros: Preserves spatial correspondence, analogous to VGG “feature map” loss

In [13]:
intermediate_output[1]

mse = F.mse_loss(intermediate_output[1], intermediate_output[1], reduction="sum")
mse

tensor(0.)

### Function

In [14]:
import torch
import torch.nn.functional as F

def dino_perceptual_loss(
    x_real,
    x_recon,
    dino_model,
    layer_ids=[11],
    mode='cls',         # 'cls', 'mean', or 'tokens'
    reduction='mean'    # or 'none'
):
    """
    Compute perceptual loss between x_real and x_recon using DINO ViT features.

    Args:
        x_real (Tensor): Original image batch [B, 3, H, W]
        x_recon (Tensor): Reconstructed image batch [B, 3, H, W]
        dino_model (nn.Module): DINO ViT model with get_intermediate_layers
        layer_ids (list[int]): Layer indices to use for perceptual comparison
        mode (str): 'cls' | 'mean' | 'tokens'
        reduction (str): 'mean' | 'sum' | 'none'

    Returns:
        Tensor: Scalar loss (or per-sample if reduction='none')
    """
    # Get intermediate layers
    with torch.no_grad():
        feats_real = dino_model.get_intermediate_layers(x_real, n=len(dino_model.blocks)+1)
        feats_recon = dino_model.get_intermediate_layers(x_recon, n=len(dino_model.blocks)+1)

    loss = 0.0

    for layer in layer_ids:
        f_real = feats_real[layer]  # [B, T, D]
        f_recon = feats_recon[layer]

        if mode == 'cls':
            v_real = f_real[:, 0]    # CLS token
            v_recon = f_recon[:, 0]

        elif mode == 'mean':
            v_real = f_real.mean(dim=1)
            v_recon = f_recon.mean(dim=1)

        elif mode == 'tokens':
            v_real = f_real
            v_recon = f_recon

        else:
            raise ValueError(f"Unknown mode: {mode}")

        loss += F.mse_loss(v_real, v_recon, reduction=reduction)

    return loss / len(layer_ids)
